# PyTorch Deep Learning Basics

This notebook demonstrates fundamental operations with PyTorch.

**Libraries:**
- [PyTorch](https://pytorch.org/) - Deep learning framework
- [TorchVision](https://pytorch.org/vision/) - Computer vision utilities
- [TorchAudio](https://pytorch.org/audio/) - Audio processing utilities

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchaudio

print(f"PyTorch version: {torch.__version__}")
print(f"TorchAudio version: {torchaudio.__version__}")

## Tensor Basics

Tensors are the fundamental data structure in PyTorch, similar to NumPy arrays but with GPU support.

In [ ]:
# Check device availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Creating tensors
x = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32)
y = torch.randn(3, 4)  # Random normal distribution
z = torch.zeros(2, 3)
ones = torch.ones(2, 2)

print(f"1D Tensor: {x}")
print(f"Random tensor shape: {y.shape}")
print(f"Zeros tensor:\n{z}")

In [ ]:
# Tensor operations
a = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
b = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)

print(f"Matrix A:\n{a}")
print(f"Matrix B:\n{b}")
print(f"\nMatrix multiplication (A @ B):\n{torch.matmul(a, b)}")
print(f"\nElement-wise multiplication (A * B):\n{a * b}")
print(f"\nSum: {a.sum()}")
print(f"Mean: {a.mean()}")

## Autograd - Automatic Differentiation

PyTorch's autograd provides automatic differentiation for all operations on tensors.

In [ ]:
# Create tensor with gradient tracking
x = torch.tensor([2.0, 3.0], requires_grad=True)

# Forward computation: y = x^2 + 3x + 1
y = x ** 2 + 3 * x + 1

# Compute gradients
y.sum().backward()

print(f"x = {x.data}")
print(f"y = x^2 + 3x + 1 = {y.data}")
print(f"dy/dx = 2x + 3 = {x.grad}")
print(f"\nVerification: 2*[2,3] + 3 = [{2*2+3}, {2*3+3}]")

## Simple Neural Network

Define a neural network using `nn.Module`.

In [ ]:
class SimpleNet(nn.Module):
    """A simple feedforward neural network."""
    
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Create model
model = SimpleNet(input_size=10, hidden_size=20, output_size=2)
print(model)

In [ ]:
# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

## Training Loop Example (XOR Problem)

Train a neural network to learn the XOR function.

In [ ]:
# XOR dataset
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

print("XOR Truth Table:")
for i in range(len(X)):
    print(f"  {int(X[i][0])} XOR {int(X[i][1])} = {int(y[i][0])}")

In [ ]:
# Simple network for XOR
class XORNet(nn.Module):
    def __init__(self):
        super(XORNet, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

xor_model = XORNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(xor_model.parameters(), lr=0.1)

print("XOR Network:")
print(xor_model)

In [ ]:
# Training loop
print("Training XOR classifier...")
losses = []

for epoch in range(1000):
    # Forward pass
    outputs = xor_model(X)
    loss = criterion(outputs, y)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}")

In [ ]:
# Test predictions
with torch.no_grad():
    predictions = xor_model(X)
    print("Predictions after training:")
    for i in range(len(X)):
        pred = predictions[i].item()
        expected = y[i].item()
        print(f"  {X[i].tolist()} -> {pred:.4f} (expected: {expected})")

## TorchVision Transforms

Image preprocessing pipelines for computer vision.

In [ ]:
# Define a transform pipeline for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print("Transform pipeline for image preprocessing:")
print("  1. Resize to 224x224")
print("  2. Convert to tensor (HWC -> CHW, scale to [0,1])")
print("  3. Normalize with ImageNet statistics")

In [ ]:
# Data augmentation transforms
augment_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])

print("Data augmentation transforms:")
print("  - Random horizontal flip")
print("  - Random rotation (+/- 15 degrees)")
print("  - Color jitter")
print("  - Random resized crop")

## TorchAudio Basics

Audio processing with TorchAudio.

In [ ]:
# Create a synthetic waveform (A4 note = 440 Hz)
sample_rate = 16000
duration = 1.0  # seconds
t = torch.linspace(0, duration, int(sample_rate * duration))
frequency = 440  # Hz (A4 note)
waveform = torch.sin(2 * torch.pi * frequency * t).unsqueeze(0)

print("Synthetic waveform created:")
print(f"  Sample rate: {sample_rate} Hz")
print(f"  Duration: {duration} seconds")
print(f"  Frequency: {frequency} Hz (A4 note)")
print(f"  Shape: {waveform.shape} (channels, samples)")

In [ ]:
# Apply Mel Spectrogram transform
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_mels=64,
)
mel_spec = mel_spectrogram(waveform)

print(f"Mel spectrogram shape: {mel_spec.shape}")
print(f"  - Channels: {mel_spec.shape[0]}")
print(f"  - Mel bins: {mel_spec.shape[1]}")
print(f"  - Time frames: {mel_spec.shape[2]}")

In [ ]:
# Other audio transforms
mfcc_transform = torchaudio.transforms.MFCC(
    sample_rate=sample_rate,
    n_mfcc=13,
)
mfcc = mfcc_transform(waveform)
print(f"MFCC shape: {mfcc.shape}")

# Spectrogram
spec_transform = torchaudio.transforms.Spectrogram()
spec = spec_transform(waveform)
print(f"Spectrogram shape: {spec.shape}")

---

## Summary

In this notebook, we covered:

1. **Tensor Basics**: Creating tensors, operations, device management
2. **Autograd**: Automatic differentiation with `requires_grad`
3. **Neural Networks**: Defining models with `nn.Module`
4. **Training Loop**: Forward pass, loss, backward pass, optimizer step
5. **TorchVision**: Image transforms and augmentation
6. **TorchAudio**: Audio waveforms, Mel spectrograms, MFCCs

For more information:
- [PyTorch Tutorials](https://pytorch.org/tutorials/)
- [TorchVision Documentation](https://pytorch.org/vision/stable/)
- [TorchAudio Documentation](https://pytorch.org/audio/stable/)